Imports

In [73]:
import pandas as pd
import numpy as np
import random

# Create data

In [24]:
df = pd.DataFrame({'Loves Popcorn':[1,1,0,0,1,1,0], 'Loves Soda':[1,0,1,1,1,0,0], 'Age':[7,12,18,35,38,50,83],'Loves Cool as Ice':[0,0,1,1,1,0,0]})

In [25]:
df

,Loves Popcorn,Loves Soda,Age,Loves Cool as Ice
0,1,1,7,0
1,1,0,12,0
2,0,1,18,1
3,0,1,35,1
4,1,1,38,1
5,1,0,50,0
6,0,0,83,0


Define feature types

In [26]:
categorical_features = ['Loves Popcorn', 'Loves Soda']
numerical_features = ['Age']
target = 'Loves Cool as Ice'

# 1. Find Feature for Root Node

## 1.1 Total Gini Impurity For each feature

1.1.0 Helper function to find gini impurity of a leaf

In [27]:
def leaf_gini_impurity(num_yes, num_no):

    total = num_yes + num_no

    gini_impurity = 1 - (num_yes/total)**2 -(num_no/total)**2

    return gini_impurity

### 1.1.1 Categorical Features

In [28]:
def binary_cat_feature_gini_impurity(feature, df):

    #Initialise counts for leaves
    leaves = {'Yes':[0, 0], 'No':[0, 0]}
    
    # When Feature is yes and target is yes
    yes_yes_count = df[(df[feature] == 1) & (df[target] == 1)].shape[0]
    leaves['Yes'][0] = yes_yes_count

    # When Feature is yes and target is no
    yes_no_count = df[(df[feature] == 1) & (df[target] == 0)].shape[0]
    leaves['Yes'][1] = yes_no_count

    # When Feature is no and target is yes
    no_yes_count = df[(df[feature] == 0) & (df[target] == 1)].shape[0]
    leaves['No'][0] = no_yes_count

    # When Feature is no and target is no
    no_no_count = df[(df[feature] == 0) & (df[target] == 0)].shape[0]
    leaves['No'][1] = no_no_count

    # Gini Impurity For Leaves
    
    # Gini Impurity Yes Leaf
    gini_impurity_yes = leaf_gini_impurity(leaves['Yes'][0], leaves['Yes'][1])

    # Gini Impurity No Leaf
    gini_impurity_no = leaf_gini_impurity(leaves['No'][0], leaves['No'][1])

    #Total Gini Impurity

    total_yes_leaf = yes_yes_count + yes_no_count

    total_no_leaf = no_yes_count + no_no_count

    gini_impurity = total_yes_leaf *gini_impurity_yes/(total_yes_leaf+total_no_leaf) + total_no_leaf *gini_impurity_no/(total_yes_leaf+total_no_leaf)

    return gini_impurity

### 1.1.2 Numerical Features

#### 1.1.2.1 Get adjacent averages on numerical features

In [64]:
def get_adjacent_averages(feature_column):

    # Feature column
    feature_column = np.sort(np.array(df[feature_column]))

    adjacent_averages = []
    for i in range(0, len(feature_column)-1):
        
        adjacent_average = (feature_column[i]+feature_column[i+1])/2

        adjacent_averages.append(adjacent_average)
        
    return adjacent_averages

#### 1.1.2.2 Get gini impurity for adjacent average

In [65]:
def numerical_gini_impurity(adjacent_average, feature_column):
    
    # Convert the feature column to binary (1 or 0)
    bool_feature_column = np.array(df[feature_column] < adjacent_average).astype(int)

    # Convert the target column to binary (assuming the target is already 1s and 0s)
    bool_target_column = np.array(df[target])

    # Initialise counts for leaves
    leaves = {'Yes': [0, 0], 'No': [0, 0]}

    # When Feature is yes (1) and target is yes (1)
    yes_yes_count = np.sum((bool_feature_column == 1) & (bool_target_column == 1))
    leaves['Yes'][0] = yes_yes_count

    # When Feature is yes (1) and target is no (0)
    yes_no_count = np.sum((bool_feature_column == 1) & (bool_target_column == 0))
    leaves['Yes'][1] = yes_no_count

    # When Feature is no (0) and target is yes (1)
    no_yes_count = np.sum((bool_feature_column == 0) & (bool_target_column == 1))
    leaves['No'][0] = no_yes_count

    # When Feature is no (0) and target is no (0)
    no_no_count = np.sum((bool_feature_column == 0) & (bool_target_column == 0))
    leaves['No'][1] = no_no_count

    # Gini Impurity For Leaves
    
    # Gini Impurity Yes Leaf
    gini_impurity_yes = leaf_gini_impurity(leaves['Yes'][0], leaves['Yes'][1])

    # Gini Impurity No Leaf
    gini_impurity_no = leaf_gini_impurity(leaves['No'][0], leaves['No'][1])

    #Total Gini Impurity

    total_yes_leaf = yes_yes_count + yes_no_count

    total_no_leaf = no_yes_count + no_no_count

    gini_impurity = total_yes_leaf *gini_impurity_yes/(total_yes_leaf+total_no_leaf) + total_no_leaf *gini_impurity_no/(total_yes_leaf+total_no_leaf)

    return gini_impurity

#### 1.1.2.3 Get optimum adjacent average (one that yields lowest gini impurity)

In [100]:
def optimum_adjacent_average(feature):

    adjacent_averages = get_adjacent_averages(feature)

    num_gini_impurities = []
    for adjacent_average in adjacent_averages:
        num_gini_impurities.append(numerical_gini_impurity(adjacent_average, feature))

    gini_impurity = min(num_gini_impurities)
    min_indices = [i for i, x in enumerate(num_gini_impurities) if x == min_element]

    rand_choice = random.randint(0, len(min_indices)-1)

    chosen_feature_index = min_indices[rand_choice] 

    chosen_adjacent_average = adjacent_averages[chosen_feature_index]

    return gini_impurity, chosen_adjacent_average

## 1.2 Get Lowest Gini Impurity Feature and hence choose roote node


In [104]:
feature_gini_impurities = {}

num_features_adjacent_average = {}

for categorical_feature in categorical_features:
    feature_gini_impurities[categorical_feature] = binary_cat_feature_gini_impurity(categorical_feature, df)

for numerical_feature in numerical_features:

    gini_impurity, chosen_adjacent_average = optimum_adjacent_average(numerical_feature)\
    
    feature_gini_impurities[numerical_feature] = gini_impurity

    num_features_adjacent_average[numerical_feature] = chosen_adjacent_average

root_node = min(feature_gini_impurities, key=feature_gini_impurities.get)

